### 1. 실행환경<br>
*시스템 :* x86_64<br> 
*OS :* Ubuntu 20.04 LTS<br>
*CPU :* Intel XeonSilver 24 core<br>
*GPU :* Nvidia A100 80G<br>
*RAM :* 120G<br>

### 2. MLflow + Triton 구조<br>
2.1 MLflow + Triton 도커 실행 시 서버 디렉토리 구조
```
aiopse/                 # ROOT Dir
|
|-- mlflow/             # mlflow 컨테이너 로컬 디렉토리와 마운트
|   |-- logs/           # mlflow 컨테이너 실행 로그 저장 위치와 마운트
|   |-- scrips/         # mlflow-triton 컨테이너 컨트롤에 필요한 스크립트 저장
|
|-- run_mlflow.sh       # 실행 스크립트
```

2.2 아티팩트 저장 방법
- 서버와 연결된 NAS 디렉토리(/artifacts) 사용
- mlflow-triton 컨테이너의 /mlflow/artfacts 디렉토리를 NAS 디렉토리와 연결

2.3 데이터베이스 연결
mlflow는 'Experiment', 'Models' 등 작업 내역(히스토리 및 메타)을 데이터베이스에 저장하며,
저장 방법은 크게 2가지.
- (기본) 내장 sqlite 사용
   - 컨테이너에 sqlite 저장 위치를 지정
- (선택) 외부 데이터베이스 연결
   - Pgsql 등 외부 데이터베에이스 접속 정보를 지정 (이 가이드는 외부 Pgsql을 사용하고 있음)

### 3. **mlflow-triton 도커 실행 스크립트 작성**<br>
: aiops/run_mlflow_triton.sh

3.1 MLflow + Triton 이미지 다운로드
- (참고) https://catalog.ngc.nvidia.com/orgs/nvidia/teams/morpheus/containers/mlflow-triton-plugin
- (명령) docker pull nvcr.io/nvidia/morpheus/mlflow-triton-plugin:latest

3.2 mlflow-triton 원본을 내 Github로 업로드
- 이미지 이름 및 태그 변경
  - docker tag nvcr.io/nvidia/morpheus/mlflow-triton-plugin:latest spansite/aiops:mlflow-triton_v2
- 업로드
  - docker push spansite/aiops:mlflow-triton_v2

### 4. mlflow-triton 컨테이너 실행
4.1 컨테이너 실행 
- sh /aiops/start_mlflow_triton.sh

4.2 컨테이너 재실행
- sh /aiops/restart_mlflow_triton.sh

4.3 컨테이너 중지
- docker stop mlflow-triton

In [ ]:
MLproject 실행 시 아래 경로에 아티팩트가 저장됨

C:\artifacts\mlflow\models\0


스테이징서버 MLflow-Triton : 2.12.1
Windows Conda 내 MLflow Client(MLproject 실행) : 3.1.4 (최신)
Experiment Docker 배포 내 MLflow Client : 2.12.1 (서버버전과 동일하게 맞춤)

In [ ]:
# 참고 명령어

# 빌드한 도커의 mlflow 버전확인
docker run --rm iris-classifier-image python -c "import mlflow; print(mlflow.__version__)"

In [ ]:
# 현재까지 진행상황

- mlflow-triton 스테이징서버에 배포 완료
- 윈도우즈에서 MLproject 를 도커로 실행완료
  -> MLflow UI에서 정상확인됨

[문제]
- 스테이징서버의 mlflow가 /artifacts 를 아티팩트 저장소로 사용하는데, /artifacts는 공유폴더(Samba, CIFS임)
- MLproject의 결과물(모델, csv)등이 스테이징서버 /artfacts/models에 저장되야하는데 Permission 오류가 발생함
- 이를 해결하려면 CIFS를 uid, gid로 강제해서 다시 마운트해야함.
- 그런데 /artfacts는 이미 팀 mlflow가 쓰고 있어서 마운트를 해제/등록하는게 이슈가 있음

-> 따라서 테스트용 mlflwo-titon의 아티팩트 저장소를 다른 디록토리로 지정한 후 아티팩트가 잘 저장되는지 확인해야함.